**Table of contents**<a id='toc0_'></a>    
- [Forward Kinematics Using Homogeneous Transformation Matrices](#toc1_)    
- [Implementation](#toc2_)    
    - [Starting pybullet](#toc2_1_1_)    
    - [Initial Setup for pybullet](#toc2_1_2_)    
    - [Generating the Robot Arm](#toc2_1_3_)    
    - [Definition of Homogeneous Transformation in 2D Plane](#toc2_1_4_)    
    - [Executing Forward Kinematics Using Homogeneous Transformation Matrices](#toc2_1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Forward Kinematics Using Homogeneous Transformation Matrices](#toc0_)

In this notebook, we will explain the procedure to implement "forward kinematics using homogeneous transformation matrices" with a 2-axis robot arm in Pybullet.

(For a manual summarizing the functions available in pybullet, refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

<br>

In forward kinematics using homogeneous transformation matrices, as shown in the video below:
- Coordinate systems are attached to each link,
- By sequentially transforming coordinates from "Link 1 coordinate system $\Sigma_{1}$" to "Link 2 coordinate system $\Sigma_{2}$" to ... to "End-effector coordinate system $\Sigma_{\mathrm{e}}$",
- The position and orientation of the "End-effector coordinate system $\Sigma_{\mathrm{e}}$" as seen from the "Link 1 coordinate system $\Sigma_{1}$" can be obtained.

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/overview_en.gif)

# <a id='toc2_'></a>[Implementation](#toc0_)
From here, we will actually implement forward kinematics using trigonometric functions for a "robot arm with 2 rotational joints" in Pybullet.

### <a id='toc2_1_1_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


### <a id='toc2_1_2_'></a>[Initial Setup for pybullet](#toc0_)

In [ ]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add paths to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the time elapsed per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position and other parameters in GUI mode
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


### <a id='toc2_1_3_'></a>[Generating the Robot Arm](#toc0_)
This time, we will generate a 2-axis robot arm `simple_2d_arm.urdf`.  
The robot is configured as shown in the diagram below (sensors are explained in `robot_arm_sensor.ipynb`). 

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# ロボットの読み込み
arm_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
arm_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf",arm_start_pos, arm_start_orientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
camera_distance = 1.5
camera_yaw = 180.0 # deg
camera_pitch = -10 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


### <a id='toc2_1_4_'></a>[Definition of Homogeneous Transformation in 2D Plane](#toc0_)



In [4]:
import numpy as np

def make_homogeneous_transformation_matrix(link_length, theta):
    """
    Calculate the homogeneous transformation matrix in a 2D plane
    
    Parameters
    ----------
    link_length : float
        Length of the link
    theta : float
        Rotation angle (rad)

    Returns
    -------
    T : numpy.ndarray
        Homogeneous transformation matrix
    """
    return np.array([[np.cos(theta), -np.sin(theta), link_length * np.cos(theta)],
                     [np.sin(theta),  np.cos(theta), link_length * np.sin(theta)],
                     [            0,              0,                         1]])

### <a id='toc2_1_5_'></a>[Executing Forward Kinematics Using Homogeneous Transformation Matrices](#toc0_)

Next, by running the code below, forward kinematics will be calculated with the angles set in `link1_angle_deg` and `link2_angle_deg`, and the calculated "end-effector position" will be displayed on the screen. (Note that the coordinates are based on the origin at the root of the robot arm (=link1 coordinate system))

Also, the angles of each joint will be set to `link1_angle_deg` and `link2_angle_deg`, so please check that the result of the forward kinematics matches the actual position of the end-effector.




In [5]:
LINK1_LENGTH = 0.5   # Length of link1 (length of link1 in "simple2d_arm.urdf")
LINK2_LENGTH = 0.55  # Length of link2 (length of link2 + force_sensor_link in "simple2d_arm.urdf")

# Changing these values will change the results ##########
# Define the rotation angles of each link
link1_angle_deg = 60
link2_angle_deg = 45
############################################

link1_angle_rad  = np.deg2rad(link1_angle_deg)
link2_angle_rad  = np.deg2rad(link2_angle_deg)

# Transformation matrix from link1 coordinate system to link2 coordinate system
H12 = make_homogeneous_transformation_matrix(LINK1_LENGTH, link1_angle_rad ) # T12: link "1" coordinate system -> link "2" coordinate system
# Transformation matrix from link2 coordinate system to end-effector coordinate system
H2e = make_homogeneous_transformation_matrix(LINK2_LENGTH, link2_angle_rad ) # T2e: link "2" coordinate system -> "e"nd effector coordinate system

# Predefine the homogeneous transformation matrix H12e that transforms from link1 coordinate system to end-effector coordinate system
H12e = H12 @ H2e # T1e: link "1" coordinate system -> "e"nd effector coordinate system

x1, y1 = 0, 0
# Use H12e to find the position of the end-effector coordinate system as seen from the link1 coordinate system
oe = H12e @ np.array([x1, y1, 1])
xe, ye = oe[0], oe[1]
print("(xe, ye)=", xe, ye)

# Display the result on the screen
text_position = [0.5, 0.0, 2.0]
life_time = 10.0 # Display duration (seconds)
pybullet.addUserDebugText(f"x, y = ({xe:.2f}, {ye:.2f})", text_position, textSize=2, lifeTime=life_time)

# Actually move the joints and check if the result of the forward kinematics matches
pybullet.resetJointState(arm_id, 0, link1_angle_rad )
pybullet.resetJointState(arm_id, 1, link2_angle_rad )
pybullet.stepSimulation()

(xe, ye)= 0.10764952519361373 0.964271906351207


()